In [1]:
import os

SCALA_VERSION = '2.12'
SPARK_VERSION = '3.1.3'
# Download Kafka Jar file, this for readStream.format("kafka"), "kafka" is a driver
# kafka driver code is part of Maven Jar file
# https://mvnrepository.com/artifact/org.apache.spark/spark-sql-kafka-0-10_2.12/3.1.3
# pyspark-shell shall download the jar file behind..
os.environ['PYSPARK_SUBMIT_ARGS'] = f'--packages org.apache.spark:spark-sql-kafka-0-10_{SCALA_VERSION}:{SPARK_VERSION} pyspark-shell'

In [2]:
import findspark
findspark.init()

In [3]:
import pyspark


from pyspark.sql import SparkSession
# spark groupBy has default setting for spark.sql.shuffle.partitions as 200
# we set to  4, should NOT be done in production 
spark = SparkSession.builder.master("local[1]")\
                            .config('spark.sql.shuffle.partitions', 4)\
                            .appName("SparkStreamingKafkaBasic").getOrCreate()

22/03/14 20:56:30 WARN Utils: Your hostname, ubuntu-virtual-machine resolves to a loopback address: 127.0.1.1; using 192.168.80.128 instead (on interface ens33)
22/03/14 20:56:30 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/opt/spark-3.1.3-bin-hadoop2.7/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ubuntu/.ivy2/cache
The jars for the packages stored in: /home/ubuntu/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2611e0db-9819-4cd3-aecf-0c40af9a6628;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.1.3 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.1.3 in central
	found org.apache.kafka#kafka-clients;2.6.0 in central
	found com.github.luben#zstd-jni;1.4.8-1 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.8.2 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 691ms :: artifacts dl 17ms
	:: modules in use:
	com.github.luben#zstd-jni;1.4.8-1 from central in [default]
	org.apache.commons#commons-pool2;2.6.2 from central

In [4]:
# read from kafka, here spark is consumer for kafka topic called test
# spark streaming works as dataframe/sql
# group.id is consumer group id
# subcribe is kafka topic
# "kafka" driver is not available by default with spark, we need to download it, we did on cell 1
kafkaDf = spark.readStream.format("kafka")\
  .option("kafka.bootstrap.servers", "localhost:9092")\
  .option("subscribe", "test")\
  .option("group.id", "wordcount-group")\
  .load()

In [5]:
kafkaDf.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [7]:
linesDf = kafkaDf.selectExpr("CAST(value AS STRING)", "timestamp" )
linesDf.printSchema() # we get only value as string

root
 |-- value: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)



In [10]:
import pyspark.sql.functions as F

wordsDf = linesDf.select(F.explode(F.split(linesDf.value," ")).alias("word") , "timestamp")

wordCountsDf = wordsDf.groupBy("word").count()
wordCountsDf.printSchema()

root
 |-- word: string (nullable = true)
 |-- count: long (nullable = false)



In [11]:
echoOnconsole = wordCountsDf\
                .writeStream\
                .outputMode("complete")\
                .format("console")\
                .start() # start the query. spark will subscribe for data

22/03/12 03:16:47 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-5de299ba-f692-4083-9f3e-245c17e5b313. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



In [12]:
import pyspark.sql.functions as F

# convert all the columns into json
# * represent all columns ie word, count, 
# struct create a structure around word, count columns
# to json convert structure to column
# value is Kafka value part of message
wordCountsToKafkaDf = wordCountsDf\
                    .selectExpr("to_json(struct(*)) as value")

wordCountsToKafkaDf.printSchema()

root
 |-- value: string (nullable = true)



In [13]:
wordCountsToKafkaDf.writeStream.format("kafka")\
                    .outputMode("complete")\
                     .option("kafka.bootstrap.servers", "localhost:9092")\
                    .option("topic", "word-counts")\
                    .option("checkpointLocation", "file:///tmp/spark")\
                    .start()
                    

22/03/12 03:19:22 WARN NetworkClient: [Producer clientId=producer-1] Error while fetching metadata with correlation id 1 : {word-counts=LEADER_NOT_AVAILABLE}


-------------------------------------------
Batch: 1
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|kafka|    1|
+-----+-----+

-------------------------------------------
Batch: 1
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|kafka|    1|
+-----+-----+



-------------------------------------------
Batch: 2
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|hello|    1|
|kafka|    1|
+-----+-----+

-------------------------------------------
Batch: 2
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|hello|    1|
|kafka|    1|
+-----+-----+



-------------------------------------------
Batch: 3
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|hello|    1|
|kafka|    2|
+-----+-----+

-------------------------------------------
Batch: 3
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|hello|    1|
|kafka|    2|
+-----+-----+



-------------------------------------------
Batch: 4
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|hello|    2|
|kafka|    3|
+-----+-----+

-------------------------------------------
Batch: 4
-------------------------------------------
+-----+-----+
| word|count|
+-----+-----+
|hello|    2|
|kafka|    3|
+-----+-----+



-------------------------------------------
Batch: 5
-------------------------------------------
+-------+-----+
|   word|count|
+-------+-----+
|  hello|    2|
|welcome|    1|
|  kafka|    4|
+-------+-----+

-------------------------------------------
Batch: 5
-------------------------------------------
+-------+-----+
|   word|count|
+-------+-----+
|  hello|    2|
|welcome|    1|
|  kafka|    4|
+-------+-----+

